In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,gc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve,auc,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import time
plt.style.use('seaborn-whitegrid')
import warnings
warnings.simplefilter("ignore")
colors = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

In [ ]:
train_trn = pd.read_csv("../input/ieee-fraud-detection/train_transaction.csv")

In [ ]:
train_id = pd.read_csv("../input/ieee-fraud-detection/train_identity.csv")

In [ ]:
test_trn  = pd.read_csv("../input/ieee-fraud-detection/test_transaction.csv")
test_id = pd.read_csv("../input/ieee-fraud-detection/test_identity.csv")

In [ ]:
df_train = train_trn.merge(train_id,on=['TransactionID'],how='left')
#print(f'Shape of identity train data: {train_id.shape}')

In [ ]:
del train_trn
del train_id

In [ ]:
id_cols = [col for col in test_id.columns if col[0]+col[1] == 'id']
rename_cols = {i:'id_'+str(i[-2]+i[-1]) for i in id_cols}
test_id = test_id.rename(columns=rename_cols)
df_test = test_trn.merge(test_id,on=['TransactionID'],how='left')
del test_trn
del  test_id

In [ ]:
df_train.drop(columns='V107',inplace=True)
df_test.drop(columns='V107',inplace=True)

In [ ]:
null_cols = [col for col in df_train.columns if df_train[col].isnull().sum() / df_train.shape[0] > 0.9]
df_train.drop(null_cols,axis=1,inplace=True)


#null_col = [col for col in df_test.columns if df_test[col].isnull().sum() / df_test.shape[0]>.9]
#df_test.drop(null_cols,axis=1,inplace=True)


In [ ]:
for i in df_train.columns:
    if df_train[i].dtypes=='object':     #filling null alues with mode for categorical variables
        df_train[i].fillna(df_train[i].mode()[0],inplace=True)

#for i in df_test.columns:
#    if df_test[i].dtypes=='object':     #filling null alues with mode for categorical variables
#        df_test[i].fillna(df_test[i].mode()[0],inplace=True)        

In [ ]:
float_col = df_train.select_dtypes(include='float64').columns
#int_col = df_train.select_dtypes(include='int64').columns
df_train[float_col] = df_train[float_col].astype('float32')

In [ ]:
for i in df_train.columns:
    if df_train[i].dtypes=='int64' or df_train[i].dtypes=="float32":     #filling null alues with mode for categorical variables
        df_train[i].fillna(df_train[i].mean(),inplace=True)

In [ ]:
#float_col = df_test.select_dtypes(include='float64').columns
#df_test[float_col] = df_test[float_col].astype('float32')

In [ ]:
#del float_col

Let's add some extra column based on the information of the existing column

In [ ]:
## transaction 
df_train['transaction_day_of_the_week'] = np.floor((df_train['TransactionDT'] / (3600 * 24) - 1) % 7)
df_train['Transaction_hour'] = np.floor(df_train['TransactionDT'] / 3600) % 24

In [ ]:
df_train['device_name'] = df_train['DeviceInfo'].str.split('/', expand=True)[0]  #categorical

In [ ]:
df_train['device_version'] = df_train['DeviceInfo'].str.split('/', expand=True)[1]  #categorical

In [ ]:
## Most of the column of id_12 - id_38 don't give any useful information without id 30, 31 and 34

df_train["device_type"] = df_train["id_30"].str.split(' ', expand = True)[0]

In [ ]:
df_train['device_version_id_30'] = df_train['id_30'].str.split(' ', expand=True)[1]   #categorical

In [ ]:
df_train['browser_id_31'] = df_train['id_31'].str.split(' ', expand=True)[0]   #categorical
df_train['version_id_31'] = df_train['id_31'].str.split(' ', expand=True)[1]   #categorical
df_train['id_34'] = df_train['id_34'].str.split(':', expand=True)[1]

In [ ]:
df_train.loc[df_train['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
df_train.loc[df_train['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
df_train.loc[df_train['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
df_train.loc[df_train['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
df_train.loc[df_train['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
df_train.loc[df_train['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
df_train.loc[df_train['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
df_train.loc[df_train['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
df_train.loc[df_train['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
df_train.loc[df_train['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'

df_train.loc[df_train['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'

df_train.loc[df_train['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
df_train.loc[df_train['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
df_train.loc[df_train['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
df_train.loc[df_train['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
df_train.loc[df_train['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
df_train.loc[df_train['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'
df_train.loc[df_train.device_name.isin(df_train.device_name.value_counts()[df_train.device_name.value_counts() < 200].index), 'device_name'] = "Others"

In [ ]:
#df_train.loc[df_train['device_name']=='Samsung'].count()

In [ ]:
df_train = df_train[df_train['TransactionAmt'] < 30000]

In [ ]:
#def detectmissing(df):
#    summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
#    summary = summary.reset_index()
#    summary['Uniques'] = df.nunique().values
    
 #   return summary

In [ ]:
#train_missing = detectmissing(df_train)
#train_missing
#test_missing = detectmissing(test)
# test_missing

Feature V107 has one unique value

In [ ]:
test_dummy = pd.get_dummies(df_test, drop_first=True)
#test_dummy.info()
float_col = test_dummy.select_dtypes(include='float64').columns
test_dummy[float_col] = test_dummy[float_col].astype("float32")

In [ ]:
del df_test

In [ ]:
train_dummy = pd.get_dummies(df_train, drop_first = True)
train_dummy.drop(columns = "TransactionID", inplace = True)

In [ ]:
del df_train

In [ ]:
#obj_cols = ['id_12','id_15', 'id_16','id_28', 'id_29','id_30', 'id_31', 'id_33', 'id_34', 'id_35', 
#            'id_36', 'id_37', 'id_38', 'device_type', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
#            'R_emaildomain', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9','device_name','device_version','OS_id_30','device_version_id_30','browser_id_31','version_id_31']

In [ ]:
#df_train.info()

In [ ]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#for i in obj_cols:
#    if i in df_train.columns:
#        df_train[i] = le.fit_transform(df_train[i].astype(str).values)

In [ ]:
#y= df_train['isFraud']
#x = df_train.drop(['isFraud','TransactionDT','TransactionID'],axis=1)

In [ ]:
#del train_dummy

In [ ]:
#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test = train_test_split(x,y,stratify = y,test_size = 0.3, random_state=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
    train_dummy.drop(columns='isFraud'), train_dummy['isFraud'], test_size=0.25, random_state=0)

In [ ]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train,label=y_train)
d_test = lgb.Dataset(X_valid, label = y_valid)

In [ ]:
#del df_train

In [ ]:
params = {'num_leaves': 250,
          'task':'train',
          'application':'classification',
          'min_child_samples': 80,
          'objective': 'binary',
          'max_depth': 15,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'random_state': 1,
          'colsample_bytree': 0.9,
          'is_unbalance':True}# unbalanced dataset 3.5%
          

              
model1 = lgb.train(params, d_train, num_boost_round= 500, valid_sets=[d_train,d_test], verbose_eval=50, early_stopping_rounds=50)

In [ ]:
y_pred1 = model1.predict(test_dummy,predict_disable_shape_check=True)

In [ ]:
#from sklearn.ensemble import RandomForestClassifier #importing the RandomForestClassifier library for our decision tree classifier
#model2 = RandomForestClassifier(max_depth=20,n_estimators=100,random_state=1)    #creating the instance of classifier with 100 trees
#model2.fit(X_train,y_train)                          #fitting our model on training data

In [ ]:
#y_pred2 = model2.predict(test_dummy)

In [ ]:
#y_pred = (y_pred1+y_pred2)/2

In [ ]:
submission = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv')
submission['isFraud'] = y_pred1  #_final
submission.to_csv('submission.csv',index=False)
submission.head()